In [10]:
%%capture
%run 02_account.ipynb
%run 03_block.ipynb

## Miner

Mining is a crucial component on any blockchain relying on proof of work (abbr. pow). It is used to reach consensus in a network of anonymous decentralized nodes.

The miner takes a new block and tries to solve a specific computational puzzle, a specific hash. This can only be done by brute force. 

In [15]:
class Miner:
    def __init__(self, acc): 
        self.acc = acc
        self.pub = acc.pub
        
    def mine(self, txs, prev_header, diff, reward, attempts=1000):
        txs.insert(0, self.coinbase(reward))
        mt = MerkleTree(txs)
        bh = Header(mt.root, prev_header.hash, prev_header.number+1, len(txs))
        bh_b = bytes(bh)
        nonce = 0
        for i in range(attempts):
            candidate   = bh_b + str(nonce).encode()
            candidate_h = sha(candidate)
            if candidate_h[:diff] == '0'*diff: break
            nonce += 1
        bh.diff   = diff
        bh.reward = reward
        bh.nonce  = nonce
        bh.miner  = self.pub
        bh.mined  = True
        return Block(bh, txs)
        
    def coinbase(self, reward): 
        return self.acc.sign(TX(self.pub, self.pub, reward, 0, self.acc.nonce))

### Genesis Block

Is the first block in the blockchain. This is how it all begins. 

In [16]:
def mine_genesis(txs):
    mt  = MerkleTree(txs)
    bh  = Header(mt.root, rh(), 0, len(txs))
    return Block(bh, txs)

gb  = mine_genesis(r_stxs(2)); print(gb)

hash:		💓 2d5d9f95f7fbac63...27f
time:		Mon Mar 29 21:27:39 2021
number:		0
prev_hash:	🔀 9a946c1c33c9f191...41e
txs_count:	2
txs_root:	🕓 ed261466747cb276...36a


### Mining

Mine a block on top of the genesis block

In [17]:
miner = Miner(acc1)
mb = miner.mine(r_stxs(10), gb.bh, 2, 100); print(mb)

hash:		🔥 bf1d13d66c07bc16...ea7
time:		Mon Mar 29 21:27:41 2021
number:		1
prev_hash:	💓 2d5d9f95f7fbac63...27f
txs_count:	11
txs_root:	🕒 eca335a01a57ebf3...524
miner:		👫 0x0557298e5D0ed7...a3C
diff:		2
reward:		100
nonce:		273
